## week03: Логистическая регрессия и анализ изображений


В этом ноутбуке предлагается построить классификатор изображений на основе логистической регрессии. 

*Забегая вперед, мы попробуем решить задачу классификации изображений используя лишь простые методы. В третьей части нашего курса мы вернемся к этой задаче.*


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

%matplotlib inline

## 1. Постановка задачи ##


**Задача**: Есть датасет [прямая ссылка](https://drive.google.com/file/d/15tOimf2QYWsMtPJXTUCwgZaOTF8Nxcsm/view?usp=sharing) ("catvnoncat.h5") состоящий из:
    - обучающей выборки из m_train изображений, помеченных "cat" (y=1) или "non-cat" (y=0)
    - тестовой выборки m_test изображений, помеченных "cat" или "non-cat"
    - каждое цветное изображение имеет размер (src_size, src_size, 3), где 3 - число каналов (RGB).
    Таким образом, каждый слой - квадрат размера src_size x src_size$.

Давайте построим простой алгоритм классификации изображений на классы "cat"/"non-cat".

Автоматическая загрузка доступна ниже.

<img src="img/LogReg_kiank.png" style="width:650px;height:400px;">

**Recap**:

Для каждого примера $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

Функция потерь:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{6}$$

In [ ]:
# Uncomment this cell to download the data

# !wget "https://downloader.disk.yandex.ru/disk/7ef1d1e30e23740a4a30799a825319154815ddc85bf689542add0a3d11ccb91c/5d7fdcb0/3dcxK38Q0fG3ui0g2gMZgKkLls8ULwVpoYNkWpBm9d24EceJ6mIoH5l3_wKkFv3PfZ0WMGYjfJULynuJkuGaug%3D%3D?uid=76549735&filename=data.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=76549735&fsize=2815580&hid=084389255415f71a92d0f1024ab741d4&media_type=compressed&tknv=v2&etag=2b348ac8eca72d223108e36b2a671210" -O data.zip
# !unzip data.zip

### 1.1 Загрузка данных и визуализация ###

In [2]:
def load_dataset():
    train_data = h5py.File("data/train_catvnoncat.h5", "r")
    train_set_x_orig = np.array(train_data["train_set_x"][:]) # признаки
    train_set_y_orig = np.array(train_data["train_set_y"][:]) # метки классов

    test_data = h5py.File("data/test_catvnoncat.h5", "r")
    test_set_x_orig = np.array(test_data["test_set_x"][:]) # признаки
    test_set_y_orig = np.array(test_data["test_set_y"][:]) # метки классов

    classes = np.array(test_data["list_classes"][:]) # the list of classes
    classes = np.array(list(map(lambda x: x.decode('utf-8'), classes)))
    
    train_set_y = train_set_y_orig.reshape(train_set_y_orig.shape[0])
    test_set_y = test_set_y_orig.reshape(test_set_y_orig.shape[0])
    return train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes

In [3]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [8]:
train_set_x_orig[0].shape

(64, 64, 3)

Цветные изображения в формате RGB представлены в виде трёхмерных numpy.array.

Порядок измерений $H \times W \times C$: $H$ - высота, $W$ - ширина и $C$ - число каналов.

Значение каждого пиксела находится в интервале $[0;255]$.

In [9]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def show_image_interact(i=0):
    f, ax = plt.subplots(1,4, figsize=(15,20), sharey=True)
    
    ax[0].imshow(train_set_x_orig[i])
    ax[0].set_title('RGB image')
    ax[1].imshow(train_set_x_orig[i][:,:,0])
    ax[1].set_title('R channel')
    ax[2].imshow(train_set_x_orig[i][:,:,1])
    ax[2].set_title('G channel')
    ax[3].imshow(train_set_x_orig[i][:,:,2])
    ax[3].set_title('B channel')
    
    print("y = {} belongs to '{}' class.".format(str(train_set_y[i]),classes[np.squeeze(train_set_y[i])]))

interact(show_image_interact,
         i=widgets.IntSlider(min=0, max=len(train_set_y)-1, step=1))

interactive(children=(IntSlider(value=0, description='i', max=208), Output()), _dom_classes=('widget-interact'…

<function __main__.show_image_interact(i=0)>

При работе с данными полезно будет сохранить размерности входных изображений для дальнейшей обработки.

In [10]:
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
src_size = train_set_x_orig.shape[1]

print ("Размер обучающей выборки: m_train = " + str(m_train))
print ("Размер тестовой выборки: m_test = " + str(m_test))
print ("Ширина/Высота каждого изображения: src_size = " + str(src_size))
print ("Размерны трёхмерной матрицы для каждого изображения: (" + str(src_size) + ", " + str(src_size) + ", 3)")
print ("Размерность train_set_x: " + str(train_set_x_orig.shape))
print ("Размерность train_set_y: " + str(train_set_y.shape))
print ("Размерность test_set_x: " + str(test_set_x_orig.shape))
print ("Размерность test_set_y: " + str(test_set_y.shape))

Размер обучающей выборки: m_train = 209
Размер тестовой выборки: m_test = 50
Ширина/Высота каждого изображения: src_size = 64
Размерны трёхмерной матрицы для каждого изображения: (64, 64, 3)
Размерность train_set_x: (209, 64, 64, 3)
Размерность train_set_y: (209,)
Размерность test_set_x: (50, 64, 64, 3)
Размерность test_set_y: (50,)


## 2. Предварительная обработка

Преобразуем входные изображения размера (num_px, num_px, 3) в вектор признаков размера (num_px $*$ num_px $*$ 3, 1), чтобы сформировать матрицы объект-признак в виде numpy-array для обучающей и тестовой выборок.

Каждой строке матрицы объект-признак соответствует входное развёрнутое в вектор-строку изображение.

Помимо этого, для предварительной обработки (препроцессинга) изображений применяют центрирование значений: из значения каждого пиксела вычитается среднее и делят полученное значение на среднеквадратичное отклонение значений пикселей всего изображения.

Однако, на практике обычно просто делят значения пикселей на 255 (максимальное значение пикселя).

Оформим эти шаги в функцию предварительной обработки

In [64]:
np.vstack(np.mean(np.ravel(train_set_x_orig).reshape((209, 64*64*3)), axis=1))

array([[ 68.24666341],
       [111.46508789],
       [133.14973958],
       [ 57.01822917],
       [ 64.0086263 ],
       [ 57.30900065],
       [ 63.30525716],
       [ 76.60058594],
       [ 95.82682292],
       [ 83.15388997],
       [131.06477865],
       [108.03198242],
       [ 82.48990885],
       [139.74739583],
       [122.53409831],
       [148.3980306 ],
       [129.62451172],
       [157.26171875],
       [120.11368815],
       [ 56.83455404],
       [112.65454102],
       [ 68.20402018],
       [166.65030924],
       [153.19254557],
       [127.44783529],
       [ 40.5012207 ],
       [ 97.6636556 ],
       [109.92643229],
       [ 75.72241211],
       [ 91.47688802],
       [ 49.84684245],
       [129.70019531],
       [106.12052409],
       [123.3030599 ],
       [ 76.        ],
       [ 75.68693034],
       [ 93.61376953],
       [158.96419271],
       [ 21.56803385],
       [ 97.09505208],
       [ 92.58789062],
       [ 40.13785807],
       [113.22680664],
       [126

In [85]:
DATA = np.ravel(train_set_x_orig).reshape((209, 64*64*3))
DATA_mean = np.vstack(np.mean(DATA, axis=1))
(DATA - DATA_mean)/np.vstack(np.sqrt(np.sum((DATA - DATA_mean)**2, axis=1) / 209))

array([[-0.12920512, -0.09390777, -0.03087677, ..., -0.1720662 ,
        -0.1720662 , -0.1720662 ],
       [ 0.18971531,  0.18073842,  0.17624997, ..., -0.06612627,
        -0.07061472, -0.06837049],
       [-0.22169297, -0.26936911, -0.2823717 , ...,  0.02102198,
         0.03402456,  0.03835876],
       ...,
       [ 0.08250746,  0.1184136 ,  0.14833538, ..., -0.09103887,
        -0.02521095,  0.10046053],
       [ 0.00416023,  0.01244294,  0.00830159, ..., -0.07038418,
        -0.06624283, -0.08694961],
       [-0.17582569, -0.10958338, -0.02678048, ..., -0.20232262,
        -0.20232262, -0.20232262]])

In [86]:
def image_preprocessing_simple(data):
    assert type(data) == np.ndarray
    assert data.ndim == 4
    
    n,h,w,c = data.shape
    data_vectorized = np.ravel(data).reshape((n, h*w*c))
    data_mean = np.vstack(np.mean(data_vectorized, axis=1))
    data_normalized = (data_vectorized - data_mean) /np.vstack(np.sqrt(np.sum((data_vectorized - data_mean)**2, axis=1) / n))
    
    return data_normalized

In [87]:
# Изменить размеры входных данных

train_set_x_vectorized = image_preprocessing_simple(train_set_x_orig)
test_set_x_vectorized = image_preprocessing_simple(test_set_x_orig)

print('Train set:')
print("Размеры train_set_x_vectorized:  {}".format(str(train_set_x_vectorized.shape)))
print("Размеры train_set_y:             {}".format(str(train_set_y.shape)))
print("Размеры классов 'cat'/'non-cat': {} / {}".format(sum(train_set_y==1), sum(train_set_y==0)))
print('Test set:')
print("Размеры test_set_x_vectorized:   {}".format(str(test_set_x_vectorized.shape)))
print("Размеры test_set_y:              {}".format(str(test_set_y.shape)))
print("Размеры классов 'cat'/'non-cat': {} / {}".format(sum(test_set_y==1), sum(test_set_y==0)))

Train set:
Размеры train_set_x_vectorized:  (209, 12288)
Размеры train_set_y:             (209,)
Размеры классов 'cat'/'non-cat': 72 / 137
Test set:
Размеры test_set_x_vectorized:   (50, 12288)
Размеры test_set_y:              (50,)
Размеры классов 'cat'/'non-cat': 33 / 17


## 3. Классификация

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

**Вопрос**: Какую метрику качества стоит использовать?

### 3.1 Построение модели

Построим  модель с параметрами по умолчанию и посмотрим, как хорошо она справится с задачей.

In [105]:
clf = LogisticRegression(random_state=0)
clf.fit(train_set_x_vectorized, train_set_y)
score = clf.score(train_set_x_vectorized, train_set_y)
print('Точность для простой модели с параметрами по умолчанию: {:.4f}'.format(score))

Точность для простой модели с параметрами по умолчанию: 1.0000


In [106]:
from sklearn.metrics import f1_score

y_predicted = clf.predict(test_set_x_vectorized)
correct_score = clf.score(test_set_x_vectorized, test_set_y)
_f1_score = f1_score(test_set_y, y_predicted)
print('acc для простой модели: {:.4f}'.format(correct_score))
print('f1_score для простой моделиЖ {:.4f}'.format(_f1_score))

acc для простой модели: 0.7200
f1_score для простой моделиЖ 0.7812


Попробуем подобрать параметры регуляризации в надежде, что это повысит точность предсказаний.

In [223]:
clf.max_iter = 1000
clf.solver = 'saga'
clf.tol =0.0001
clf.C = 2
clf.penalty='elasticnet'
clf.l1_ratio=0
clf.fit(train_set_x_vectorized, train_set_y)
score = clf.score(test_set_x_vectorized, test_set_y)
y_pred = clf.predict(test_set_x_vectorized)
_f1_score = f1_score(test_set_y, y_pred)

In [224]:
print('Оптимальные параметры: {}'.format(5000))
print('Наилучшее значение метрики качества f1: {}'.format(_f1_score))
print('Наилучшее значение метрики качества: {}'.format(score))

Оптимальные параметры: 5000
Наилучшее значение метрики качества f1: 0.7812499999999999
Наилучшее значение метрики качества: 0.72


Обучим модель с оптимальными параметрами на всей обучающей выборке и посмотрим на метрики качества:

In [226]:
best_clf = LogisticRegression()
best_clf.fit(train_set_x_vectorized, train_set_y)

y_predicted = best_clf.predict(test_set_x_vectorized)
metric_score = f1_score(y_predicted, test_set_y)
print('Optimal model hyperparameters accuracy score: {:.4f}'.format(metric_score))

Optimal model hyperparameters accuracy score: 0.7812


### 3.2 Анализ ошибок

In [227]:
is_outlier = (y_predicted != test_set_y)
test_outliers_x, test_outliers_y, predicted_y = test_set_x_orig[is_outlier], test_set_y[is_outlier], y_predicted[is_outlier]

In [228]:
def show_image_outliers(i=0):
    f = plt.figure(figsize=(5,5))
    plt.imshow(test_outliers_x[i])
    plt.title('RGB image')
    
    fmt_string = "Sample belongs to '{}' class, but '{}' is predicted'"
    print(fmt_string.format(classes[test_outliers_y[i]], classes[predicted_y[i]]))

interact(show_image_outliers,
         i=widgets.IntSlider(min=0, max=len(test_outliers_y)-1, step=1))


interactive(children=(IntSlider(value=0, description='i', max=13), Output()), _dom_classes=('widget-interact',…

<function __main__.show_image_outliers(i=0)>

**Вопрос**: Как по-вашему можно повысить точность? Каким недостатком обладает данный подход к классификации?

### 3.3 Модель с аугментациями

Как можно увеличить количество данных для обучения?

Сформировать новые примеры из уже имеющихся!

Например, можно пополнить class 'cat' обучающей выборки [зеркально отображёнными](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fliplr.html) изображениями котов.

In [ ]:
def augment_sample(src, label):
    

def image_preprocessing_augment(data, labels):
    assert type(data) == np.ndarray
    assert data.ndim == 4
    
    ## ВАШ КОД ##
    
    
    data_augmented = 
    labels_augmented = 
    ## ВАШ КОД ЗАКАНЧИВАЕТСЯ ЗДЕСЬ ##
    
    n,h,w,c = data_augmented.shape
    data_vectorized = data_augmented.reshape(n, -1) # <ваш код>
    data_normalized = data_vectorized / 255
    
    return data_normalized, labels_augmented

In [ ]:
train_set_x_augmented, train_set_y_augmented = image_preprocessing_augment(train_set_x_orig, train_set_y)

In [ ]:
clf = LogisticRegression(solver='liblinear')
clf.fit(train_set_x_augmented, train_set_y_augmented)
y_pred = clf.predict(test_set_x_vectorized)
print('F-мера для модели с аугментациями: {:.4f}'.format(f1_score(y_pred, test_set_y)))

## 4. Проверьте работу классификатора на своей картинке

Библиотека [OpenCV](https://opencv.org) для работы с изображениями для [python](https://pypi.org/project/opencv-python/):

`pip install opencv-python`

Вместе с contrib-модулями:

`pip install opencv-contrib-python`


In [ ]:
import cv2

# Путь к картинке на вашем ПК
fname = "cat-non-cat.jpg"
# Считываем картинку через scipy
src = cv2.cvtColor(cv2.imread((fname)), cv2.COLOR_BGR2RGB)
src_resized = cv2.resize(src, (src_size,src_size), interpolation=cv2.INTER_LINEAR).reshape(1, src_size*src_size*3)
my_image_predict = clf.predict(src_resized)[0]

plt.imshow(src)
print("Алгоритм говорит, что это '{}': {}".format(my_image_predict, classes[my_image_predict]))